# Лабораторна робота номер 2
### ФБ-24 Довгаль Анна

1. Створити env в якому будуть встановлені всі необхідні бібліотеки та
налаштування для данної лабораторної роботи


```conda
conda create --name adenv python
conda activate adenv
conda install pandas
conda install urllib.request
```

2. Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура
має бути автоматизована, параметром процедури має бути індекс
(номер) області. При зберіганні файлу до його імені потрібно додати
дату та час завантаження;

In [ ]:
import urllib
import datetime

def get_csv(i):
    url='https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2024&type=Mean'.format(i)
    wp = urllib.request.urlopen(url)
    text = wp.read()
    now = datetime.datetime.now()
    date_and_time = now.strftime("%d-%m-%Y_%H-%M-%S")
    out = open('.\\csv\\NOAA_ID_'+str(i)+'_'+date_and_time+'.csv','wb')
    out.write(text)
    out.close()

for i in range(1,28):
    get_csv(i)

3. Зчитати завантажені текстові файли у фрейм (детальніше про роботу
із фреймами буде розказано у подальших лабораторних роботах).
Імена стовбців фрейму мають бути змістовними та легкими для
сприйняття (не повинно бути спеціалізованих символів, пробілів
тощо). Ця задача має бути реалізована у вигляді окремої процедури,
яка на вхід приймає шлях до директорії, в якій зберігаються файли;

4. Реалізувати процедуру, яка змінить індекси областей, які використані
на порталі NOAA на дані:

In [44]:
import pandas as pd
import os

def convert_ids(file, df):
    correct_ids = {1 : 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 12: 26, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 20: 27, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
    parts = file.split("_")
    old_index = int(parts[2])
    new_index = correct_ids.get(old_index)
    df['area'] = new_index


def to_df(path):
    dfs=[]
    for file in os.listdir(path):
        headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
        df = pd.read_csv(os.path.join(path, file), header = 1, names = headers)

        convert_ids(file, df)

        dfs.append(df)

    df = pd.concat(dfs, ignore_index=True)

    df = df.drop(df.loc[df['VHI'] == -1].index)
    df.replace({"<tt><pre>1982": "1982"}, inplace=True)
    df.drop(df[df['Year'] == '</pre></tt>'].index, inplace=True)
    df = df.drop(columns=['empty'])
    df['Year'] = pd.to_numeric(df['Year'])
    return df
    


df = to_df('.\\csv\\')
df.head()


,Year,Week,SMN,SMT,VCI,TCI,VHI,area
0,1982,1.0,0.059,258.24,51.11,48.78,49.95,21
1,1982,2.0,0.063,261.53,55.89,38.20,47.04,21
2,1982,3.0,0.063,263.45,57.30,32.69,44.99,21
3,1982,4.0,0.061,265.10,53.96,28.62,41.29,21
4,1982,5.0,0.058,266.42,46.87,28.57,37.72,21


In [29]:
df.tail()

,Year,Week,SMN,SMT,VCI,TCI,VHI,area
60350,2024,5.0,0.101,275.13,58.55,8.77,33.66,20
60351,2024,6.0,0.110,276.43,60.83,9.08,34.96,20
60352,2024,7.0,0.120,277.97,61.35,10.45,35.90,20
60353,2024,8.0,0.130,279.15,60.18,13.87,37.02,20
60354,2024,9.0,0.142,280.64,59.61,16.63,38.12,20


5. Реалізувати процедури для формування вибірок наступного виду
(включаючи елементи аналізу):
- Ряд VHI для області за вказаний рік, пошук екстремумів (min та
max);
- Ряд VHI за вказаний діапазон років для вказаних областей;
виявити роки, протягом яких екстремальні посухи торкнулися
більше вказаного відсотка областей по Україні (20% областей -
5 областей з 25);
- Аналогічно для помірних посух

In [ ]:
def get_extremum(area, year):
    min_v = df[(df["area"] == area) & (df["Year"] == year)]['VHI'].min()
    max_v = df[(df["area"] == area) & (df["Year"] == year)]['VHI'].max()
    print('min: ', min_v, '\nmax: ', max_v)

selected = df[(df["area"] == 7) & (df["Year"] == 2005)]['VHI']
#selected.head()
get_extremum(1, 2000)

In [45]:
filtered = df[(df['Year'] >= 2000) & (df['Year'] <= 2023) & (df['area'].isin([2, 6, 7]))]
print(filtered[['Year', 'area', 'VHI']])

       Year  area    VHI
32254  2000     6  36.69
32255  2000     6  40.87
32256  2000     6  43.27
32257  2000     6  44.39
32258  2000     6  44.24
...     ...   ...    ...
40208  2023     7  34.82
40209  2023     7  35.22
40210  2023     7  35.57
40211  2023     7  36.56
40212  2023     7  37.99

[3684 rows x 3 columns]


In [ ]:
def ext_droughts(pers):
    total = 27  
    years = df[df['VHI'] <= 15].groupby('Year')['area'].nunique() #рахує кількість областей на кожен рік з посухою
    #print(years)
    overpers = years[years > total * (pers / 100)]
    print(f"Роки з екстремальними посухами, які торкнулися більше {pers}% областей по Україні:\n{overpers}")

ext_droughts(20)

In [ ]:
def mild_droughts(pers):
    total = 27  
    years = df[(df['VHI'] <= 35) & (df['VHI'] > 15)].groupby('Year')['area'].nunique() #рахує кількість областей на кожен рік з посухою
    #print(years)
    overpers = years[years > total * (pers / 100)]
    print(f"Роки з помірними посухами, які торкнулися більше {pers}% областей по Україні:\n{overpers}")

mild_droughts(70)

In [ ]:
df.to_csv("lab2.csv")